# Data and styles

This notebook loads all data required for the exploration, analysis, and modeling, then stores it for use by other notebooks using [storemagic](https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html). Make sure to run this notebook before using any of the other notebooks in this repository.

In [ ]:
import os
from pathlib import Path
import warnings

import earthpy as et
import earthpy.plot as ep
import geopandas as gpd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rasterio.enums import Resampling
import rioxarray as rxr
import xarray as xr

from ea_drought_burn.config import CRS
from ea_drought_burn.utils import (
    concat_arrays,
    copy_xr_metadata,
    load_nifc_fires,
    open_raster,
    plot_bands,
    plot_rgb,
    reproject_match
)


os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data", "woolsey-fire"))

## Load data

### Woolsey Fire perimeter

The Woolsey Fire perimeter is provided courtesy of the National Interagency Fire Center (2019). There are no known restrictions on the perimeter shapefile.

In [ ]:
# Load the Woolsey Fire perimeter
woolsey_fire = gpd.read_file(os.path.join("shapefiles",
                                          "nifc_woolsey_perimeter",
                                          "2018-CAVNC-091023.shp")).to_crs(CRS)

# Store data for use in other notebooks
%store woolsey_fire

### Reprojection raster

All raster data is reprojeted to a common projection, defined by the `reproj_to` varaible in the next cell. Currently, all rasters are reprojected to match the MTBS classified dNBR raster. See **MTBS burn severity data** below for more information about MTBS data.

In [ ]:
# Load raster to use for reprojections
path = os.path.join("mtbs-burn-severity",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr6.tif")
reproj_to = open_raster(path, crs=CRS, crop_bound=woolsey_fire.envelope)
reproj_to = reproj_to.where(reproj_to != -9999, np.nan)
reproj_to = reproj_to.rio.write_nodata(np.nan)

# Store data for use in other notebooks
%store reproj_to

### Sentinel-2 imagery (ESA)

Sentinel-2 (ESA) imagery from before and after the Woolsey Fire was originally collected by the European Space Agency and was provided courtesy of the U.S. Geological Survey. The data below was collected on 31 Oct 2018 (roughly one week before the Woolsey Fire started) and 15 Dec 2018 (three to four weeks after the Woolsey Fire was contained) and has 30 m resolution.

In [ ]:
# Load pre-fire data from Sentinel-2 (ESA)
path = os.path.join("sentinel-2-imagery",
                    "3_aligned",
                    "L1C_T11SLT_A008633_20181031T184032.tif")
s2_prefire = reproject_match(open_raster(path), reproj_to)

# Load post-fire data from Sentinel-2 (ESA)
path = os.path.join("sentinel-2-imagery",
                    "3_aligned",
                    "L1C_T11SLT_A018185_20181215T184316.tif")
s2_postfire = reproject_match(open_raster(path), reproj_to)

# Name each band in the Sentinel-2 imagery
ultrablue = s2_prefire[0]
blue = s2_prefire[1]        # L8 band 1
green = s2_prefire[2]       # L8 band 2
red = s2_prefire[3]         # L8 band 3
nir_705 = s2_prefire[4]     # L8 band 4
nir_740 = s2_prefire[5]
nir_783 = s2_prefire[6]
nir_842 = s2_prefire[7]     # L8 band 5
nir_865 = s2_prefire[8]     # L8 band 5
swir_940 = s2_prefire[9]
swir_1375 = s2_prefire[10]  # L8 band 9
swir_1610 = s2_prefire[11]  # L8 band 6
swir_2190 = s2_prefire[12]  # L8 band 7

# Calculate pre-fire spectral indices
s2_ndvi = (nir_842 - red) / (nir_842 + red)
s2_ndwi = (nir_865 - swir_1610) / (nir_865 + swir_1610)
s2_ndmi = (nir_842 - swir_1610) / (nir_842 + swir_1610)
s2_savi = 1.5 * (nir_842 - red) / (nir_842 + red + 0.5)

# Store data for use in other notebooks
%store s2_prefire
%store s2_postfire
%store s2_ndvi
%store s2_ndwi
%store s2_ndmi
%store s2_savi

### USGS topographic data

USGS topographic data, including elevation, aspect, slope, and hillshades, for the Woolsey Fire area. The utility used to produce this data is not known, but there are no known restrictions on USGS topographic data. Data was collected at 15 m resolution but has been reproject to 15.6 m resolution to match the NASA AVIRIS data.

In [ ]:
# Load USGS elevation data
path = os.path.join("usgs-terrain", "usgs_2016_18_merge_rs15m_fix.tif")
elevation = open_raster(path)
elevation = reproject_match(elevation, reproj_to)

# Load USGS slope data
path = os.path.join("usgs-terrain", "usgs_2016_18_merge_rs15m_fix_slope.tif")
slope = open_raster(path)
slope = reproject_match(slope, reproj_to)

# Load USGS aspect data
path = os.path.join("usgs-terrain", "usgs_2016_18_merge_rs15m_fix_aspect.tif")
aspect = open_raster(path)
aspect = reproject_match(aspect, reproj_to)

# Calculate folded aspect
folded_aspect = np.absolute(180 - np.absolute(aspect - 225))

# Store data for use in other notebooks
%store elevation
%store slope
%store aspect
%store folded_aspect

### MTBS burn severity data

A set of normalized, field-validated burn severity maps for the Woolsey Fire created by the Monitoring Trends in Burn Severity Program (MTBS), an interagency program managed by the USGS and USDA (Eidenshink et al., 2007). There are no known restrictions on MTBS data. The data below has 30 m resolution and was calculated using Sentinel-2 imagery from 15 Dec 2017 and 15 Dec 2018.

In [ ]:
# Load MTBS dNBR
path = os.path.join("mtbs-burn-severity",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr.tif")
mtbs_dnbr = open_raster(path, crs=CRS)
mtbs_dnbr = mtbs_dnbr.rio.write_nodata(np.nan)
mtbs_dnbr = mtbs_dnbr.where(mtbs_dnbr != -9999, np.nan)
mtbs_dnbr /= 1000  # MTBS pixels are scaled by 1000
mtbs_dnbr = reproject_match(mtbs_dnbr, reproj_to)

# Load MTBS classified dNBR
path = os.path.join("mtbs-burn-severity",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr6.tif")
mtbs_cl_dnbr = open_raster(path, crs=CRS, crop_bound=woolsey_fire.geometry)
mtbs_cl_dnbr = mtbs_cl_dnbr.rio.write_nodata(np.nan)
mtbs_cl_dnbr = mtbs_cl_dnbr.where(mtbs_cl_dnbr != -9999, np.nan)
mtbs_cl_dnbr = mtbs_cl_dnbr.where(mtbs_cl_dnbr != 6, np.nan)
mtbs_cl_dnbr = reproject_match(mtbs_cl_dnbr, reproj_to)

# Store data for use in other notebooks
%store mtbs_dnbr
%store mtbs_cl_dnbr

### Santa Monica Mountains vegetation and climate

Vegetation and climate data for the Santa Monica Mountains, California for 2013-2016. This file combines data from the NASA AVIRIS sensor (15.6 m resolution) and the PRISM Gridded Climate dataset (4 km resolution). The compiled data was provided by E. Natasha Stavros and was originally created by Dagit et al. (2017) and Foster et al. (2017).

In [ ]:
# Load SMM stack
smm_stack = open_raster(
    os.path.join("aviris-climate-vegetation", "SMMDroughtstack.dat"),
    crs=CRS
)
smm_stack = smm_stack.rio.write_nodata(np.nan)
smm_stack = smm_stack.where((smm_stack >= -1e38) & (smm_stack != -9999))
smm_stack = reproject_match(smm_stack, reproj_to)

# Store data for use in other notebooks
%store smm_stack

In [ ]:
# Make calculations based on FAL
fal = smm_stack[1:5]

# Calculate fraction dead
fdd = 1 - fal

# Calculate year-on-year difference in FAL (dFAL)
dfal = concat_arrays([fal[i+1] - fal[i] for i in range(3)])

# Store data for use in other notebooks
%store fal
%store fdd
%store dfal

In [ ]:
# Threshold is the point below which a pixel is considered dead. The field
# constraint on this value is poor and was calculated only for oaks (0.5431).
threshold = 0.5

# Calculate dead pixels
dead = xr.where(fal < threshold, True, False)

# Calculate years dead. This is the point at which a pixel permanently dropped
# below the live/dead threshold.
years_dead = xr.where(fal[3] <= threshold, 2, 0)
for i, xda in enumerate([fal[2], fal[1], fal[0]]):
    mask = (
        xr.where(years_dead > 0, True, False)
        * xr.where(xda <= threshold, True, False)
    )
    years_dead = xr.where(mask.values, i + 3, years_dead)
    
years_dead = years_dead.where(smm_stack[0] < 6, np.nan)

# Store data for use in other notebooks
%store dead
%store years_dead

### Live fuel moisture content

Live fuel moisture content (LFMC) estimates how likely vegetation is to ignite based on how much moiture it contains. The data below was calculated using a neural-network model trained using both optical and microwave radiation (Rao et al., 2020) and was downloaded using the instructions on https://github.com/kkraoj/lfmc_from_sar. The data was calculated for 1 Nov 2018, a week before the start of the Woolsey Fire, and was calculated to 30 m resolution.

In [ ]:
# Load LFMC data from 2018-11-01
lfmc = open_raster(
    os.path.join("_pending", "lfmc.tif"),
    crs=CRS
)
lfmc.where(lfmc != 0, np.nan)
lfmc = reproject_match(lfmc, reproj_to)

# Store data for use in other notebooks
%store lfmc

### Recent fires (2000-2018)

Perimeters for fires from 2000-2018 that intersect the Woolsey Fire scar are courtesy of the National Interagency Fire Center (2019). There are no known restrictions on the perimeter shapefiles.

In [ ]:
# Load recent fires in the study area
path = os.path.join("shapefiles",
                    "nifc_related_perimeters",
                    "RELATED_FIRE_PERIMTRS_2000_2018_DD83.shp")
related_fires = load_nifc_fires(path, crs=CRS)
related_fires.sort_values(["perimeterd"], inplace=True)

# Build array showing when areas of the scar last burned
last_burned = xr.where(smm_stack[0], -9999, -9999)
last_burned.rio.write_nodata(-9999, inplace=True)

woolsey_date = related_fires.iloc[-1].perimeterd
for i, row in related_fires.iterrows():
    days = (woolsey_date - row.perimeterd).days
    if days:
    
        # Create a mask based on last_burned with a nodata value of False
        mask = xr.where(last_burned, True, True)
        mask = mask.rio.write_nodata(0) 

        # Update the mask
        mask = mask.rio.clip([row.geometry], drop=False)

        last_burned = copy_xr_metadata(last_burned,
                                       xr.where(mask, days, last_burned))

# Use burned/unburned instead of days since last fire. The latter feels more
# useful, but not clear how to handle pixels that have not burned recently.
last_burned = xr.where(last_burned == -9999, 0, 1)

# Store data for use in other notebooks
%store last_burned

### PRISM grid

The PRISM grid raster is used to aggregate other data to the exact grid used by PRISM.

In [ ]:
# Load the PRISM grid
prism_grid = open_raster(
    os.path.join("masks", "prism_grid.tif"),
    crs=CRS,
    crop_bound=woolsey_fire.envelope,
    masked=False
)
prism_grid = reproject_match(prism_grid, reproj_to)

# Store data for use in other notebooks
%store prism_grid

### Feature and label data lookup

The lookup dictionary is used to provide a consistent way to access data used for modeling the Woolsey Fire across multiple notebooks. Notebooks generally use this dictionary instead of calling the stored variables directly. Some data, like fire perimeters, is not included.

In [ ]:
# Create lookup for all data available for use by the random-forest model
all_data = {
    # Vegetation
    "Community": smm_stack[0],
    "FAL": fal,
    "FDD": fdd,
    "dFAL": dfal,
    "Dead": dead,
    "Years Dead": years_dead,
    "Burned (2000-2018)": last_burned,
    
    # Pre- and post-fire Sentinel-2 imagery
    "Sentinel-2 Prefire": s2_prefire,
    "Sentinel-2 Postfire": s2_postfire,
    
    # Pre-fire imagery and spectral indicdes
    "LFMC": lfmc,
    "NDVI": s2_ndvi,
    "NDMI": s2_ndmi,
    "NDWI": s2_ndwi,
    "SAVI": s2_savi,
    
    # Topography
    "Elevation": elevation,
    "Aspect": aspect,
    "Folded Aspect": folded_aspect,
    "Slope": slope,
    
    # Climate
    "Days Precipitation": smm_stack[5:9],
    "Max VPD": smm_stack[9:13],
    "Minimum Temperature": smm_stack[13:17],
    "Heat Days Over 95": smm_stack[17:21],
    "Cumulative Precipitation": smm_stack[21:25],
    
    # Burn severity
    "dNBR": mtbs_dnbr,
    "Classified dNBR": mtbs_cl_dnbr,
}


# Verify that all data has the same shape
shapes = {
    "prism_grid": prism_grid.shape[-2:],
}
shapes.update({k: v.shape[-2:] for k, v in all_data.items()})
if len(set([tuple(s) for s in shapes.values()])) != 1:
    display(shapes)
    raise ValueError("Shapes do not match")

# Verify that all data has the same bounds
bounds = {
    "prism_grid": prism_grid.rio.bounds(),
}
bounds.update({k: v.rio.bounds() for k, v in all_data.items()})
if len(set([s for s in bounds.values()])) != 1:
    display(bounds)
    raise ValueError("Bounds do not match")
    
# Standardize nodata value to np.nan if dtype is float and nodata is None
for key, xda in all_data.items():
    if xda.rio.nodata is None and xda.dtype in (float, np.float32, np.float64):    
        all_data[key] = xda.rio.write_nodata(np.nan)

# Store data for use in other notebooks
%store all_data

## Set styles

This section defines colors and labels for complex plots that appear in multiple notebooks.

In [ ]:
# Define color map and labels for vegetation community data
cmap_vegetation = ListedColormap([
    "black",
    "tab:green",
    "tab:olive",
    "tab:cyan",
    "tab:red",
    "tab:blue",
    "lightgray"
])

labels_vegetation = [
    "No data",
    "Annual grass",
    "Chaparral",
    "Coastal sage scrub",
    "Oak woodland",
    "Riparian",
    "Substrate"
]

# Store data for use in other notebooks
%store cmap_vegetation
%store labels_vegetation

In [ ]:
# Define color map and labels for MTBS dNBR data
cmap_dnbr = ListedColormap([
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
])

labels_dnbr = [
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
    "Increased Greenness"
]

# Store data for use in other notebooks
%store cmap_dnbr
%store labels_dnbr

## Set ready variable

The ready variable can be used to check if this notebook has been run and all data is available.

In [ ]:
woolsey_data_ready = True
%store woolsey_data_ready

## References

+ Dagit R, Contreras S, Daukiss R, Spyrka A, Quelly N, Foster K, Nickmeyer A,
  Rousseau B, Chang E. 2017. How can we save our native trees? Drought and
  Invasive Beetle impacts on Wildland Trees and Shrublands in the Santa Monica
  Mountains. Final Report for Los Angeles County Contract CP-03-44. Avalable
  from:
  https://www.rcdsmm.org/wp-content/uploads/2016/04/Drought-and-Invasive-Beetle-impacts-RCDSMM-1.2.18.pdf.
  
  
+ Eidenshink J, Schwind B, Brewer K et al. 2007. A Project for Monitoring
  Trends in Burn Severity. Fire Ecol 3, 3â€“21.
  doi:[10.4996/fireecology.0301003](https://doi.org/10.4996/fireecology.0301003).
  
  
+ Foster K, Queally N, Nickmeyer A, Rousseau N. 2017A. Appendix: Santa Monica
  Mountains Ecological Forecasting II: Utilizing NASA Earth Observations to
  Determine Drought Dieback and Insect-related Damage in the Santa Monica
  Mountains, California. Avalable from:
  https://www.rcdsmm.org/wp-content/uploads/2016/04/Drought-and-Tree-Appendices_12.15.17.pdf.


+ Krishna R, Williams AP, Flefil JF, Konings AG. 2020. SAR-enhanced mapping of
  live fuel moisture content. Remote Sens Environ. 245.
  doi:[10.1016/j.rse.2020.111797](https://doi.org/10.1016/j.rse.2020.111797).
  
  
+ National Interagency Fire Center. 2019. Historic Perimeters Combined
  2000-2018. Available from:
  https://data-nifc.opendata.arcgis.com/datasets/historic-perimeters-combined-2000-2018.
  
  
+ PRISM Climate Group. Oregon State University https://prism.oregonstate.edu,
  created October 2017.